In [10]:
# default_exp metaflow

In [11]:
# export


import os
import subprocess
from pathlib import Path, PosixPath
from typing import Iterable

from fastcore.script import call_parse
from nbdev.export import Config, find_default_export, nbglob, read_nb
from sciflow.data_handler import extract_param_meta
from sciflow.params import params_as_dict
from sciflow.parse_module import FuncDetails, extract_steps

# Sciflow Notebook to MetaFlow Flow

> Converts from a `sciflow` format notebook to a `metaflow` flow. 

Supported features:

* Linear/sequential DAGs
* Simple `Parameters`

In [12]:
%load_ext autoreload
%autoreload 2

In [57]:
nb_path = Path(os.path.join("test", "test_export.ipynb"))
nb = read_nb(nb_path)
module_name = find_default_export(nb["cells"]).replace(".", "/")
test_module = os.path.join(Config().path("lib_path"), f"{module_name}.py")

In [14]:
# export


def titleize(name):
    return name.title().replace("_", "")

In [15]:
assert titleize("snake_case") == "SnakeCase"

In [16]:
# export


def rename_steps_for_metaflow(steps):
    for i, step in enumerate(steps):
        if i == 0:
            step.name = "start"
        if i == len(steps) - 1:
            step.name = "end"

In [17]:
steps = extract_steps(test_module)

In [18]:
no_steps = extract_steps(os.path.join(Config().path("lib_path"), f"_nbdev.py"))
assert len(no_steps) == 0

In [19]:
assert ["first", "preprocess", "train", "last"] == [step.name for step in steps]
rename_steps_for_metaflow(steps)
assert ["start", "preprocess", "train", "end"] == [step.name for step in steps]

In [20]:
# export


def indent_multiline(multiline_text, indent=1):
    lines = multiline_text.strip().split("\n")
    spaces = "".join(["    " for _ in range(indent)])
    for i in range(len(lines)):
        prefix = spaces if i > 0 else spaces + '"""'
        lines[i] = prefix + lines[i]
    return "\n".join(lines) + '"""'

In [21]:
text = """
Some text
:param param: text
"""
assert '    """Some text\n    :param param: text"""' == indent_multiline(text)

In [55]:
# Test Returns

In [56]:
nb_path = Path(os.path.join("test", "test_top2vec.ipynb"))
nb = read_nb(nb_path)
module_name = find_default_export(nb["cells"]).replace(".", "/")
test_module = os.path.join(Config().path("lib_path"), f"{module_name}.py")
steps = extract_steps(test_module)

In [61]:
steps[1]

FuncDetails(name='preprocess', docstring=None, args='dremio_access,model_level,min_date,traffic_percent', has_return=False, code='\n\ndef preprocess(dremio_access, model_level, min_date, traffic_percent):\n    data = get_utterances(dremio_access, model_level, min_date, traffic_percent)\n    button_filter = get_button_responses_filter(dremio_access)\n    user_texts = data[~data.Utterance.isin(button_filter)].copy()\n    documents = user_texts.Utterance.tolist()\n    return documents\n\n')

In [22]:
# export


def nb_to_metaflow(nb_path: Path, flow_path: Path, silent=True):
    nb = read_nb(nb_path)
    lib_name = Config().lib_name
    module_name = find_default_export(nb["cells"])
    if not module_name:
        return
    module_name = module_name
    path_sep_module_name = module_name.replace(".", "/")
    nb_name = os.path.basename(nb_path)
    exported_module = os.path.join(
        Config().path("lib_path"), f"{path_sep_module_name}.py"
    )
    steps = extract_steps(exported_module)
    if len(steps) == 0:
        return
    orig_step_names = [step.name for step in steps]
    if len(steps) == 1:
        steps.append(FuncDetails("end", None, None, False, "pass"))
    params = params_as_dict(nb_path)
    if len(params) == 0:
        print(f"No params cell found for: {os.path.basename(nb_path)}")
    flow_class_name = f"{titleize(extract_module_only(module_name))}Flow"
    rename_steps_for_metaflow(steps)
    write_module_to_file(
        flow_path,
        flow_class_name,
        lib_name,
        module_name,
        orig_step_names,
        steps,
        params,
    )
    if not silent:
        print(
            f"Converted {nb_name} to {flow_class_name} in: {os.path.basename(flow_path)}"
        )

In [23]:
pkg_module_name = "test.module"
module_name = "module"
path_sep_module_name = module_name.replace(".", "/")

In [24]:
# export


def extract_module_only(package_module_name):
    module_name = package_module_name
    if "." in module_name:
        package_name, module_name = module_name.split(".")
    return module_name

In [25]:
assert "module" == extract_module_only(module_name)
assert "module" == extract_module_only(pkg_module_name)

In [26]:
#  export


def write_module_to_file(
    flow_path: Path,
    flow_class_name: str,
    lib_name: str,
    module_name: str,
    orig_step_names: Iterable[str],
    steps: Iterable[FuncDetails],
    params: dict,
):
    if not os.path.exists(flow_path.parent):
        os.mkdir(flow_path.parent)
    fq_module_name = f"{lib_name}.{module_name}"
    param_meta = extract_param_meta(fq_module_name, params)
    with open(flow_path, "w") as flow_file:
        flow_file.write("# SCIFLOW GENERATED FILE - DO NOT EDIT\n")
        has_mf_param = any((p.has_metaflow_param for p in param_meta.values()))
        has_json_param = any((p.is_json_type for p in param_meta.values()))
        mf_params_import = "from metaflow import FlowSpec, step"
        if has_mf_param:
            mf_params_import += ", Parameter"
        if has_json_param:
            mf_params_import += ", JSONType"
            flow_file.write("import json\n")
        flow_file.write(mf_params_import + "\n")
        flow_file.write(f"from {fq_module_name} import {', '.join(orig_step_names)}\n")
        flow_file.write(f"from {fq_module_name} import {', '.join(params.keys())}\n")
        flow_file.write(f"\n\nclass {flow_class_name}(FlowSpec):\n")
        single_indent = "    "
        write_params(flow_file, param_meta, single_indent)

        flow_file.write("\n")
        write_steps(flow_file, steps, orig_step_names, param_meta, single_indent)

        flow_file.write('if __name__ == "__main__":\n')
        flow_file.write(f"{single_indent}{flow_class_name}()")

In [27]:
#  export


def write_params(flow_file, param_meta, single_indent):
    for param in param_meta.keys():
        if param_meta[param].is_scalar:
            flow_file.write(
                f"{single_indent}{param} = Parameter('{param}', default={param})\n"
            )
        elif param_meta[param].is_json_type:
            flow_file.write(
                f"{single_indent}{param} = Parameter('{param}', default=json.dumps({param}), type=JSONType)\n"
            )
        elif param_meta[param].instance_type == PosixPath:
            flow_file.write(
                f"{single_indent}{param} = Parameter('{param}', default=str({param}))\n"
            )

In [28]:
nb_path = os.path.join(Path(".").resolve(), "test", "test_data_handling.ipynb")
params = params_as_dict(nb_path)
param_meta = extract_param_meta("sciflow.test.test_data_handling", params)

In [29]:
assert any((p.has_metaflow_param for p in param_meta.values()))
assert any((p.is_json_type for p in param_meta.values()))

In [30]:
# export


def format_arg(arg, param_meta):
    result = arg
    if arg in param_meta and param_meta[arg].has_metaflow_param:
        result = "self." + arg
    return result


def write_steps(flow_file, steps, orig_step_names, param_meta, single_indent):
    for i, step in enumerate(steps):
        flow_file.write(f"{single_indent}@step\n")
        flow_file.write(f"{single_indent}def {step.name}(self):\n")
        if step.docstring:
            flow_file.write(f"{indent_multiline(step.docstring, 2)}\n")
        # Check for padded step
        if i < len(orig_step_names):
            flow_step_args = ""
            if len(step.args) > 0:
                flow_step_args = ", ".join(
                    [format_arg(a, param_meta) for a in step.args.split(",")]
                )
            flow_file.write(
                f"{single_indent}{single_indent}{orig_step_names[i]}({flow_step_args})\n"
            )
        else:
            flow_file.write(f"{single_indent}{single_indent}pass\n")
        if i < len(steps) - 1:
            flow_file.write(
                f"{single_indent}{single_indent}self.next(self.{steps[i+1].name})\n"
            )
        flow_file.write("\n")

In [31]:
# export


def get_module_name(nb_path):
    nb = read_nb(nb_path)
    module_name = find_default_export(nb["cells"])
    return module_name

In [32]:
flow_path = Path(
    os.path.join(
        Path(".").resolve(),
        "test",
        "flows",
        f"{get_module_name(nb_path).split('.')[-1]}.py",
    )
)

In [33]:
assert (
    Path("/home/jovyan/git/sciflow/nbs/test/flows/test_data_handling.py") == flow_path
)

In [34]:
nb_to_metaflow(nb_path, flow_path, silent=False)

Converted test_data_handling.ipynb to TestDataHandlingFlow in: test_data_handling.py


# Ignore notebooks without Sciflow steps

In [35]:
nb_to_metaflow("packaging.ipynb", flow_path, silent=False)

In [36]:
# export


def generate_flows(config: Config):
    flows_dir = config.path("flows_path")
    nb_paths = nbglob(recursive=True)
    for nb_path in nb_paths:
        flow_module_name = os.path.basename(nb_path).replace("ipynb", "py")
        nb_to_metaflow(
            nb_path, Path(os.path.join(flows_dir, flow_module_name)), silent=False
        )

In [37]:
generate_flows(Config(cfg_name="test/settings.ini"))

Converted test_export.ipynb to TestExportFlow in: test_export.py


INFO:gensim.summarization.textcleaner:'pattern' package found; tag filters are available for English


Converted test_top2vec.ipynb to TestTop2VecFlow in: test_top2vec.py
Converted test_module.ipynb to TestModuleFlow in: test_module.py
Converted test_data_handling.ipynb to TestDataHandlingFlow in: test_data_handling.py


In [38]:
# export


@call_parse
def sciflow_generate():
    generate_flows(Config())

In [48]:
# export


def check_flows(config, flow_command="show"):
    flow_results = {}
    flows_dir = config.path("flows_path")
    for flow_path in os.listdir(flows_dir):
        flow_name = os.path.basename(flow_path)
        if flow_path.endswith(".py"):
            ret_code, output = check_flow(
                flows_dir, flow_path, flow_command=flow_command
            )
            flow_results[flow_name] = ret_code, output
            if ret_code == 0:
                print(f"Flow: {flow_name} verified")
            else:
                print(f"Flow: {flow_name} verification failed\nDetails:\n{output}")

In [49]:
# export


def prep_mf_env():
    if "USER" not in os.environ:
        try:
            os.environ["USER"] = os.environ["GIT_COMMITTER_NAME"]
        except KeyError:
            raise EnvironmentError(
                "Metaflow requires a known user for tracked execution. Add USER or GIT_COMMITTER_NAME to Jupyter environment variables"
            )

In [50]:
# export


def run_shell_cmd(script):
    pipe = subprocess.Popen(
        "%s" % script, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True
    )
    output = pipe.communicate()[0]
    return pipe, output.decode("utf-8").strip()

In [46]:
# export


def check_flow(flows_dir, flow_module, flow_command="show"):
    prep_mf_env()
    script = f"python '{os.path.join(flows_dir, flow_module)}' {flow_command}"
    pipe, output = run_shell_cmd(script)
    return pipe.returncode, output

In [47]:
check_flows(Config(cfg_name="test/settings.ini"))

Flow: test_module.py verified
Flow: test_data_handling.py verified
Flow: test_top2vec.py verified
Flow: test_export.py verified


In [51]:
check_flows(Config(cfg_name="test/settings.ini"), "--no-pylint run")

Flow: test_module.py verified
Flow: test_data_handling.py verified
Flow: test_top2vec.py verification failed
Details:
Metaflow 2.2.7 executing TestTop2VecFlow for user:e02079
Validating your flow...
    The graph looks good!
2021-04-10 07:06:38.528 Workflow starting (run-id 1618038398524406):
2021-04-10 07:06:38.539 [1618038398524406/start/1 (pid 1444)] Task is starting.
2021-04-10 07:06:46.143 [1618038398524406/start/1 (pid 1444)] Task finished successfully.
2021-04-10 07:06:46.155 [1618038398524406/preprocess/2 (pid 1463)] Task is starting.
2021-04-10 07:06:52.869 [1618038398524406/preprocess/2 (pid 1463)] <flow TestTop2VecFlow step preprocess> failed:
2021-04-10 07:06:52.870 [1618038398524406/preprocess/2 (pid 1463)]     Internal error
2021-04-10 07:06:52.870 [1618038398524406/preprocess/2 (pid 1463)] Traceback (most recent call last):
2021-04-10 07:06:52.870 [1618038398524406/preprocess/2 (pid 1463)]   File "/home/jovyan/envs/discovery/lib/python3.7/site-packages/metaflow/cli.py", 

In [44]:
# export


@call_parse
def sciflow_check_flows():
    check_flows(Config())

In [52]:
# export


@call_parse
def sciflow_run_flows():
    check_flows(Config(), "--no-pylint run")